In [142]:
!pip install noisereduce

In [143]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
import noisereduce as nr
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [147]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your spectrogram images
train_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/train'
validation_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

#train_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/train'
#validation_directory = r'/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/test'

#reserved_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023.zip (Unzipped Files)/random10_prep_data/reserved'

batchSize = 2

# Image dimensions
#img_width, img_height = 884, 322
img_width, img_height = 322, 884
# Rescale the images by dividing every pixel in every image by 255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
reserved_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directory and prepare them for training
train_generator = train_datagen.flow_from_directory(
    #train_directory,
    directory = train_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 91825)

validation_generator = validation_datagen.flow_from_directory(
    #validation_directory,
    directory = validation_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size = batchSize,
    class_mode='categorical',
    shuffle = True,
    seed = 18232)

#reserved_generator = reserved_datagen.flow_from_directory(
#    directory = reserved_directory,
#    target_size=(img_height, img_width),
#    color_mode = "grayscale",
#    batch_size=16,
#    class_mode='categorical',
#    shuffle = False)

Found 1836 images belonging to 10 classes.
Found 572 images belonging to 10 classes.


In [149]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow import keras
# Define the CNN model
model = Sequential()

# Convolutional layer with 32 filters, a kernel size of 3x3, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(322, 884, 1) ) )
model.add(MaxPooling2D((2, 2)))

# Adding another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((5, 5)))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((4, 4)))

#Adding dropout layer to control overfitting
model.add(Dropout(0.25))
# Adding dense softmax layer
model.add(Dense(128, activation='softmax'))

# Adding layer with tanh acitvation function to improve accuracy
model.add(Conv2D(128, (3, 3), activation='tanh'))
#varying the size of the max pooling kernel has helepd the model
model.add(MaxPooling2D((4, 4)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Fully connected layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

#several layers added


# Dropout for regularization
model.add(Dropout(0.15))

# Output layer with a unit for each class and softmax activation
model.add(Dense(10, activation='softmax'))

# Compile the model

opti = keras.optimizers.Adam(lr = 0.02)
model.compile(optimizer = opti, #optimizer = 'adam'
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_117 (Conv2D)         (None, 320, 882, 32)      320       
                                                                 
 max_pooling2d_111 (MaxPool  (None, 160, 441, 32)      0         
 ing2D)                                                          
                                                                 
 conv2d_118 (Conv2D)         (None, 158, 439, 64)      18496     
                                                                 
 max_pooling2d_112 (MaxPool  (None, 31, 87, 64)        0         
 ing2D)                                                          
                                                                 
 conv2d_119 (Conv2D)         (None, 27, 83, 64)        102464    
                                                                 
 max_pooling2d_113 (MaxPool  (None, 6, 20, 64)       

In [150]:
# Train the model

fitModel = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batchSize,
    epochs = 20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batchSize)


Epoch 1/20
918/918 [==============================] - 38s 38ms/step - loss: 1.1727 - accuracy: 0.6781 - val_loss: 1.1508 - val_accuracy: 0.6416
Epoch 2/20
918/918 [==============================] - 38s 41ms/step - loss: 0.8139 - accuracy: 0.7456 - val_loss: 1.1763 - val_accuracy: 0.6783
Epoch 3/20
918/918 [==============================] - 34s 37ms/step - loss: 0.6878 - accuracy: 0.7881 - val_loss: 1.2216 - val_accuracy: 0.6241
Epoch 4/20
918/918 [==============================] - 34s 37ms/step - loss: 0.6252 - accuracy: 0.7947 - val_loss: 1.2884 - val_accuracy: 0.6958
Epoch 5/20
918/918 [==============================] - 34s 37ms/step - loss: 0.5425 - accuracy: 0.8230 - val_loss: 1.2782 - val_accuracy: 0.7185
Epoch 6/20
918/918 [==============================] - 34s 37ms/step - loss: 0.4815 - accuracy: 0.8361 - val_loss: 1.3906 - val_accuracy: 0.6941
Epoch 7/20
918/918 [==============================] - 34s 37ms/step - loss: 0.4235 - accuracy: 0.8540 - val_loss: 1.4128 - val_accuracy:

In [151]:
test_directory = '/content/drive/MyDrive/P2_DeepLearning/birdclef-2023/random10_prep_data/test'

test_datagen = ImageDataGenerator(rescale=1./255, validation_split = 0.9625)

test_generator = test_datagen.flow_from_directory(
    directory = test_directory,
    target_size=(img_height, img_width),
    color_mode = "grayscale",
    batch_size= batchSize,
    class_mode='categorical',
    subset = 'validation',
    shuffle = False) #since it's test, don't set shuffle to true


Found 544 images belonging to 10 classes.


In [152]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batchSize)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


272/272 [==============================] - 6s 21ms/step - loss: 2.3776 - accuracy: 0.7243
Test accuracy: 0.7242646813392639
Test loss: 2.377568483352661


In [153]:
from sklearn.metrics import classification_report, confusion_matrix
import random
# # Predict classes
#predictions = model.predict(test_generator, steps=test_generator.samples // 32)
predictions = model.predict(test_generator, steps=test_generator.samples // batchSize)
predicted_classes = np.argmax(predictions, axis=1)

# # Get true labels
#true_classes = random.sample(test_generator.classes, 544)
true_classes = test_generator.classes
#scores report
report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys(), zero_division = 0)

print(report)


272/272 [==============================] - 6s 22ms/step
              precision    recall  f1-score   support

     afbfly1       0.24      0.21      0.23        28
     afdfly1       0.73      0.75      0.74        36
     chespa1       0.25      1.00      0.40         1
     chewea1       1.00      0.33      0.50         9
     grywrw1       0.90      0.76      0.82        49
     luebus1       0.71      0.65      0.68        31
     nobfly1       0.43      0.26      0.32        35
     rocmar2       0.26      0.31      0.28        26
     somgre1       0.81      0.91      0.86       304
     somtit4       0.42      0.20      0.27        25

    accuracy                           0.72       544
   macro avg       0.58      0.54      0.51       544
weighted avg       0.71      0.72      0.71       544



In [154]:
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)


[[  6   0   0   0   1   0   0  12   9   0]
 [  3  27   0   0   0   0   0   4   2   0]
 [  0   0   1   0   0   0   0   0   0   0]
 [  1   0   0   3   0   1   0   0   4   0]
 [  0   0   0   0  37   0   0   0  10   2]
 [  1   0   0   0   1  20   0   0   9   0]
 [  0   4   1   0   0   0   9   4  17   0]
 [  5   6   1   0   0   1   4   8   0   1]
 [  8   0   0   0   0   5   6   3 278   4]
 [  1   0   1   0   2   1   2   0  13   5]]
